In [5]:
import cv2
from keras.models import model_from_json
import numpy as np
import time
import webbrowser
import threading

# Load the pre-trained model
json_file = open("emotionsdetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotionsdetector.h5")

# Load the face cascade classifier
haar_file = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

# Function to extract features from the image
def extract_features(image):
    image = np.array(image)
    image = image.reshape(1, 48, 48, 1)
    return image / 255.0

# Define emotion labels
labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Function to open HTML file in a new window
def open_html_file():
    webbrowser.open_new("file:///C:/Users/asims/Desktop/web/Emotion%20Detection.html")

# Open webcam feed
webcam = cv2.VideoCapture(0)

# Function to handle camera feed and emotion prediction
def camera_feed():
    while True:
        ret, im = webcam.read()
        if not ret:
            break

        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        if len(faces) > 0:
            for (p, q, r, s) in faces:
                image = gray[q:q + s, p:p + r]
                cv2.rectangle(im, (p, q), (p + r, q + s), (255, 0, 0), 2)
                image = cv2.resize(image, (48, 48))
                img = extract_features(image)
                pred = model.predict(img)
                prediction_label = labels[pred.argmax()]

                # Display emotion prediction
                cv2.putText(im, f'Emotion: {prediction_label}', (p - 10, q - 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255))

            # Display the camera feed
            cv2.imshow("Camera Feed", im)

        # Check for key presses to exit the loop
        key = cv2.waitKey(1)
        if key == 27:  # ESC key
            break

# Create threads for camera feed and HTML file opening
camera_thread = threading.Thread(target=camera_feed)
html_thread = threading.Thread(target=open_html_file)

# Start both threads
camera_thread.start()
html_thread.start()

# Wait for both threads to finish
camera_thread.join()
html_thread.join()

# Release the webcam
webcam.release()

# Close all OpenCV windows
cv2.destroyAllWindows()


1/1 [==============================] - 0s 52ms/step
